In [448]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Get Data

In [532]:
init_data = pd.read_csv("./merged_data.csv")

In [533]:
select_columns = ['Total.Cup.Points', 'Species','Country.of.Origin','Processing.Method', 'Aroma', 'Flavor', 'Aftertaste','Acidity','Body', 'Balance', 'Uniformity', 'Moisture', 'altitude_mean_meters']

In [534]:
init_data  = init_data[select_columns]

In [535]:
init_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Total.Cup.Points      1339 non-null   float64
 1   Species               1339 non-null   object 
 2   Country.of.Origin     1338 non-null   object 
 3   Processing.Method     1169 non-null   object 
 4   Aroma                 1339 non-null   float64
 5   Flavor                1339 non-null   float64
 6   Aftertaste            1339 non-null   float64
 7   Acidity               1339 non-null   float64
 8   Body                  1339 non-null   float64
 9   Balance               1339 non-null   float64
 10  Uniformity            1339 non-null   float64
 11  Moisture              1339 non-null   float64
 12  altitude_mean_meters  1109 non-null   float64
dtypes: float64(10), object(3)
memory usage: 136.1+ KB


# Processing Data / Preparation 

In [536]:
init_data['Species'].unique()

array(['Arabica', 'Robusta'], dtype=object)

In [537]:
init_data['Country.of.Origin'].unique()

array(['Ethiopia', 'Guatemala', 'Brazil', 'Peru', 'United States',
       'United States (Hawaii)', 'Indonesia', 'China', 'Costa Rica',
       'Mexico', 'Uganda', 'Honduras', 'Taiwan', 'Nicaragua',
       'Tanzania, United Republic Of', 'Kenya', 'Thailand', 'Colombia',
       'Panama', 'Papua New Guinea', 'El Salvador', 'Japan', 'Ecuador',
       'United States (Puerto Rico)', 'Haiti', 'Burundi', 'Vietnam',
       'Philippines', 'Rwanda', 'Malawi', 'Laos', 'Zambia', 'Myanmar',
       'Mauritius', 'Cote d?Ivoire', nan, 'India'], dtype=object)

In [538]:
init_data['Processing.Method'].unique()

array(['Washed / Wet', nan, 'Natural / Dry', 'Pulped natural / honey',
       'Semi-washed / Semi-pulped', 'Other'], dtype=object)

In [539]:
def find_null(data) :
    null_data = data[data.isnull().any(axis=1)]
    return null_data

In [540]:
find_null(init_data)

Total.Cup.Points  Species Country.of.Origin Processing.Method  Aroma  \
2                89.75  Arabica         Guatemala               NaN   8.42   
5                88.83  Arabica            Brazil     Natural / Dry   8.58   
6                88.75  Arabica              Peru      Washed / Wet   8.42   
7                88.67  Arabica          Ethiopia               NaN   8.25   
8                88.42  Arabica          Ethiopia               NaN   8.67   
...                ...      ...               ...               ...    ...   
1331             80.17  Robusta             India               NaN   7.67   
1334             78.75  Robusta           Ecuador               NaN   7.75   
1335             78.08  Robusta           Ecuador               NaN   7.50   
1337             75.08  Robusta             India     Natural / Dry   7.42   
1338             73.75  Robusta           Vietnam     Natural / Dry   6.75   

      Flavor  Aftertaste  Acidity  Body  Balance  Uniformity  Moisture  \
2       8.50        8.42     8.42  8.33     8.42       10.00      0.00   
5       8.42        8.42     8.50  8.25     8.33       10.00      0.11   
6       8.50        8.33     8.50  8.25     8.25       10.00      0.11   
7       8.33        8.50     8.42  8.33     8.50       10.00      0.03   
8       8.67        8.58     8.42  8.33     8.42        9.33      0.03   
...      ...         ...      ...   ...      ...         ...       ...   
1331    7.67        7.50     7.33  7.58     7.50       10.00      0.00   
1334    7.58        7.33     7.58  5.08     7.83       10.00      0.00   
1335    7.67        7.75     7.75  5.17     5.25       10.00      0.00   
1337    6.83        6.75     7.17  7.25     7.00        9.33      0.10   
1338    6.67        6.50     6.83  6.92     6.83        9.33      0.12   

      altitude_mean_meters  
2                   1700.0  
5                      NaN  
6                      NaN  
7                   1635.0  
8                   1635.0  
...                    ...  
1331                 750.0  
1334                   NaN  
1335                  40.0  
1337                   NaN  
1338                   NaN  

[326 rows x 13 columns]

In [541]:
init_data = init_data.dropna()

In [542]:
init_data['Country.of.Origin'].unique()

array(['Ethiopia', 'United States', 'China', 'Costa Rica', 'Mexico',
       'Brazil', 'Uganda', 'Taiwan', 'Kenya', 'Thailand', 'Colombia',
       'Panama', 'Guatemala', 'Papua New Guinea', 'El Salvador',
       'Indonesia', 'Tanzania, United Republic Of', 'Honduras', 'Japan',
       'Nicaragua', 'Ecuador', 'United States (Puerto Rico)', 'Haiti',
       'Burundi', 'Vietnam', 'Philippines', 'Rwanda', 'Malawi', 'Laos',
       'Zambia', 'Myanmar', 'Cote d?Ivoire', 'Peru', 'India'],
      dtype=object)

In [543]:
init_data['Processing.Method'].unique()

array(['Washed / Wet', 'Natural / Dry', 'Pulped natural / honey',
       'Semi-washed / Semi-pulped', 'Other'], dtype=object)

In [544]:
init_data.describe()

Total.Cup.Points        Aroma       Flavor   Aftertaste      Acidity  \
count       1013.000000  1013.000000  1013.000000  1013.000000  1013.000000   
mean          82.115577     7.567897     7.514294     7.386772     7.527196   
std            2.626653     0.300706     0.323467     0.332248     0.307302   
min           59.830000     5.080000     6.170000     6.170000     5.250000   
25%           81.170000     7.420000     7.330000     7.170000     7.330000   
50%           82.420000     7.580000     7.500000     7.420000     7.500000   
75%           83.580000     7.750000     7.670000     7.580000     7.670000   
max           90.580000     8.750000     8.830000     8.670000     8.750000   

              Body      Balance   Uniformity     Moisture  \
count  1013.000000  1013.000000  1013.000000  1013.000000   
mean      7.505656     7.500622     9.869516     0.094689   
std       0.272702     0.340476     0.444271     0.043684   
min       6.330000     6.080000     6.000000     0.000000   
25%       7.330000     7.330000    10.000000     0.100000   
50%       7.500000     7.500000    10.000000     0.110000   
75%       7.670000     7.670000    10.000000     0.120000   
max       8.500000     8.580000    10.000000     0.170000   

       altitude_mean_meters  
count           1013.000000  
mean            1813.965837  
std             9067.972349  
min                1.000000  
25%             1100.000000  
50%             1310.640000  
75%             1600.000000  
max           190164.000000

# Split X and Y

In [545]:
Y = init_data['Total.Cup.Points']
init_X = init_data.iloc[:,1:]

In [546]:
init_X.head()

Species Country.of.Origin Processing.Method  Aroma  Flavor  Aftertaste  \
0  Arabica          Ethiopia      Washed / Wet   8.67    8.83        8.67   
1  Arabica          Ethiopia      Washed / Wet   8.75    8.67        8.50   
3  Arabica          Ethiopia     Natural / Dry   8.17    8.58        8.42   
4  Arabica          Ethiopia      Washed / Wet   8.25    8.50        8.25   
9  Arabica          Ethiopia     Natural / Dry   8.08    8.58        8.50   

   Acidity  Body  Balance  Uniformity  Moisture  altitude_mean_meters  
0     8.75  8.50     8.42        10.0      0.12                2075.0  
1     8.58  8.42     8.42        10.0      0.12                2075.0  
3     8.42  8.50     8.25        10.0      0.11                2000.0  
4     8.50  8.42     8.33        10.0      0.12                2075.0  
9     8.50  7.67     8.42        10.0      0.10                1822.5

In [547]:
std = StandardScaler()

In [548]:
numberic_col = ['Aroma' ,'Flavor' ,'Aftertaste' ,'Acidity' ,'Balance' ,'Uniformity' ,'Moisture' ,'altitude_mean_meters']

In [549]:
x_std= std.fit_transform(init_X[numberic_col])
x_outlier_checking = pd.DataFrame(x_std,columns=numberic_col)
x_outlier_checking.head()

Aroma    Flavor  Aftertaste   Acidity   Balance  Uniformity  Moisture  \
0  3.666864  4.069526    3.864168  3.981118  2.701608    0.293848  0.579691   
1  3.933036  3.574640    3.352249  3.427644  2.701608    0.293848  0.579691   
2  2.003288  3.296267    3.111346  2.906727  2.202060    0.293848  0.350663   
3  2.269460  3.048824    2.599428  3.167186  2.437141    0.293848  0.579691   
4  1.703844  3.296267    3.352249  3.167186  2.701608    0.293848  0.121636   

   altitude_mean_meters  
0              0.028801  
1              0.028801  
2              0.020526  
3              0.028801  
4              0.000942

In [550]:
x_outlier_checking.describe()

Aroma        Flavor    Aftertaste       Acidity       Balance  \
count  1.013000e+03  1.013000e+03  1.013000e+03  1.013000e+03  1.013000e+03   
mean  -4.188160e-15  2.745309e-15 -4.289209e-15 -4.260714e-15 -7.888940e-15   
std    1.000494e+00  1.000494e+00  1.000494e+00  1.000494e+00  1.000494e+00   
min   -8.277615e+00 -4.157951e+00 -3.664050e+00 -7.413937e+00 -4.174521e+00   
25%   -4.920771e-01 -5.700286e-01 -6.527627e-01 -6.420184e-01 -5.013753e-01   
50%    4.026742e-02 -4.421240e-02  1.000592e-01 -8.854429e-02 -1.827508e-03   
75%    6.058834e-01  4.816037e-01  5.818651e-01  4.649298e-01  4.977203e-01   
max    3.933036e+00  4.069526e+00  3.864168e+00  3.981118e+00  3.171770e+00   

         Uniformity      Moisture  altitude_mean_meters  
count  1.013000e+03  1.013000e+03          1.013000e+03  
mean  -6.729289e-16 -5.479877e-16          1.007064e-16  
std    1.000494e+00  1.000494e+00          1.000494e+00  
min   -8.714121e+00 -2.168642e+00         -2.000294e-01  
25%    2.938483e-01  1.216357e-01         -7.877378e-02  
50%    2.938483e-01  3.506634e-01         -5.553330e-02  
75%    2.938483e-01  5.796911e-01         -2.360743e-02  
max    2.938483e-01  1.724830e+00          2.078117e+01

In [551]:
fig = px.box(x_outlier_checking)
fig.show()

# Correlation Detect

In [552]:
#sns.pairplot(x_outlier_checking ,hue="Aroma")

In [553]:
X_std = x_outlier_checking
X_corr = X_std.corr()

In [554]:
fig = px.imshow(X_corr, text_auto=True ,aspect="auto" )
fig.show()

In [555]:
lower_cor = pd.DataFrame(np.tril(X_corr, -1), columns = X_corr.columns)
lower_cor

Aroma    Flavor  Aftertaste   Acidity   Balance  Uniformity  Moisture  \
0  0.000000  0.000000    0.000000  0.000000  0.000000    0.000000  0.000000   
1  0.730360  0.000000    0.000000  0.000000  0.000000    0.000000  0.000000   
2  0.678193  0.848149    0.000000  0.000000  0.000000    0.000000  0.000000   
3  0.603052  0.751411    0.689565  0.000000  0.000000    0.000000  0.000000   
4  0.610976  0.734293    0.764381  0.645033  0.000000    0.000000  0.000000   
5  0.134722  0.222691    0.241899  0.192677  0.242988    0.000000  0.000000   
6 -0.081144 -0.118622   -0.166376 -0.101806 -0.213424   -0.001482  0.000000   
7 -0.019391 -0.012992   -0.026866  0.003164 -0.016469   -0.012309  0.018695   

   altitude_mean_meters  
0                   0.0  
1                   0.0  
2                   0.0  
3                   0.0  
4                   0.0  
5                   0.0  
6                   0.0  
7                   0.0

In [556]:
to_drop = [column for column in lower_cor if any(lower_cor[column] > abs(0.80) )]
to_drop

['Flavor']

In [557]:
x_norm = x_outlier_checking.drop(to_drop ,axis=1)
x_norm.head()

Aroma  Aftertaste   Acidity   Balance  Uniformity  Moisture  \
0  3.666864    3.864168  3.981118  2.701608    0.293848  0.579691   
1  3.933036    3.352249  3.427644  2.701608    0.293848  0.579691   
2  2.003288    3.111346  2.906727  2.202060    0.293848  0.350663   
3  2.269460    2.599428  3.167186  2.437141    0.293848  0.579691   
4  1.703844    3.352249  3.167186  2.701608    0.293848  0.121636   

   altitude_mean_meters  
0              0.028801  
1              0.028801  
2              0.020526  
3              0.028801  
4              0.000942

# Prepare X (get dumies)

In [559]:
# get non-numerric features
categorical_features = init_X.select_dtypes(include=object).columns.tolist()
categorical_features

['Species', 'Country.of.Origin', 'Processing.Method']

In [560]:
X_categorical= pd.get_dummies(init_X[categorical_features], columns = categorical_features, drop_first=True)

In [573]:
X_categorical = X_categorical.reset_index(drop=True)
X_categorical

Species_Robusta  Country.of.Origin_Burundi  Country.of.Origin_China  \
0                   0                          0                        0   
1                   0                          0                        0   
2                   0                          0                        0   
3                   0                          0                        0   
4                   0                          0                        0   
...               ...                        ...                      ...   
1008                1                          0                        0   
1009                1                          0                        0   
1010                1                          0                        0   
1011                1                          0                        0   
1012                1                          0                        0   

      Country.of.Origin_Colombia  Country.of.Origin_Costa Rica  \
0                              0                             0   
1                              0                             0   
2                              0                             0   
3                              0                             0   
4                              0                             0   
...                          ...                           ...   
1008                           0                             0   
1009                           0                             0   
1010                           0                             0   
1011                           0                             0   
1012                           0                             0   

      Country.of.Origin_Cote d?Ivoire  Country.of.Origin_Ecuador  \
0                                   0                          0   
1                                   0                          0   
2                                   0                          0   
3                                   0                          0   
4                                   0                          0   
...                               ...                        ...   
1008                                0                          0   
1009                                0                          0   
1010                                0                          0   
1011                                0                          0   
1012                                0                          0   

      Country.of.Origin_El Salvador  Country.of.Origin_Ethiopia  \
0                                 0                           1   
1                                 0                           1   
2                                 0                           1   
3                                 0                           1   
4                                 0                           1   
...                             ...                         ...   
1008                              0                           0   
1009                              0                           0   
1010                              0                           0   
1011                              0                           0   
1012                              0                           0   

      Country.of.Origin_Guatemala  ...  Country.of.Origin_Thailand  \
0                               0  ...                           0   
1                               0  ...                           0   
2                               0  ...                           0   
3                               0  ...                           0   
4                               0  ...                           0   
...                           ...  ...                         ...   
1008                            0  ...                           0   
1009                            0  ...                           0   
1010                            0  .

## Concat X

In [574]:
x_norm

Aroma  Aftertaste   Acidity   Balance  Uniformity  Moisture  \
0     3.666864    3.864168  3.981118  2.701608    0.293848  0.579691   
1     3.933036    3.352249  3.427644  2.701608    0.293848  0.579691   
2     2.003288    3.111346  2.906727  2.202060    0.293848  0.350663   
3     2.269460    2.599428  3.167186  2.437141    0.293848  0.579691   
4     1.703844    3.352249  3.167186  2.701608    0.293848  0.121636   
...        ...         ...       ...       ...         ...       ...   
1008  0.339711    1.093784 -0.088544  0.233254    0.293848  0.350663   
1009 -0.225905   -0.411860  0.985847  0.967883    0.293848  0.808719   
1010  0.040267    0.100059  0.985847 -0.001828    0.293848 -2.168642   
1011  1.171499    0.100059 -0.349003 -0.236909   -1.214987 -2.168642   
1012 -0.791521   -0.652763 -0.349003 -0.971538   -1.214987 -2.168642   

      altitude_mean_meters  
0                 0.028801  
1                 0.028801  
2                 0.020526  
3                 0.028801  
4                 0.000942  
...                    ...  
1008             -0.089807  
1009             -0.117390  
1010             -0.117390  
1011              0.130858  
1012             -0.112425  

[1013 rows x 7 columns]

In [605]:
X = pd.concat([x_norm, X_categorical], join="inner" ,axis=1)[:-1]
X

Aroma  Aftertaste   Acidity   Balance  Uniformity  Moisture  \
0     3.666864    3.864168  3.981118  2.701608    0.293848  0.579691   
1     3.933036    3.352249  3.427644  2.701608    0.293848  0.579691   
2     2.003288    3.111346  2.906727  2.202060    0.293848  0.350663   
3     2.269460    2.599428  3.167186  2.437141    0.293848  0.579691   
4     1.703844    3.352249  3.167186  2.701608    0.293848  0.121636   
...        ...         ...       ...       ...         ...       ...   
1007  0.339711    0.581865  0.985847  1.467431    0.293848 -0.107392   
1008  0.339711    1.093784 -0.088544  0.233254    0.293848  0.350663   
1009 -0.225905   -0.411860  0.985847  0.967883    0.293848  0.808719   
1010  0.040267    0.100059  0.985847 -0.001828    0.293848 -2.168642   
1011  1.171499    0.100059 -0.349003 -0.236909   -1.214987 -2.168642   

      altitude_mean_meters  Species_Robusta  Country.of.Origin_Burundi  \
0                 0.028801                0                          0   
1                 0.028801                0                          0   
2                 0.020526                0                          0   
3                 0.028801                0                          0   
4                 0.000942                0                          0   
...                    ...              ...                        ...   
1007             -0.089807                1                          0   
1008             -0.089807                1                          0   
1009             -0.117390                1                          0   
1010             -0.117390                1                          0   
1011              0.130858                1                          0   

      Country.of.Origin_China  ...  Country.of.Origin_Thailand  \
0                           0  ...                           0   
1                           0  ...                           0   
2                           0  ...                           0   
3                           0  ...                           0   
4                           0  ...                           0   
...                       ...  ...                         ...   
1007                        0  ...                           0   
1008                        0  ...                           0   
1009                        0  ...                           0   
1010                        0  ...                           0   
1011                        0  ...                           0   

      Country.of.Origin_Uganda  Country.of.Origin_United States  \
0                            0                                0   
1                            0                                0   
2                            0                                0   
3                            0                                0   
4                            0                                0   
...                        ...                              ...   
1007                         0                                0   
1008                         0                                0   
1009                         0                                0   
1010                         0                                0   
1011                         0                                1   

      Country.of.Origin_United States (Puerto Rico)  \
0                                                 0   
1                                                 0   
2                                                 0   
3                                                 0   
4                                                 0   
...                                             ...   
1007                                              0   
1008                                              0   
1009                                              0   
1010                                              0   
1011                                              0 

# Prepare Y 

In [595]:
Bean_Grade = [1,2,3]

In [596]:
rating_pctile = np.percentile( Y, [75, 90])

In [597]:
Y = np.where(Y < rating_pctile[0], 1, Y)

In [598]:
Y = np.where((Y >= rating_pctile[0]) & (Y <= rating_pctile[1]), 2, Y)

In [599]:
Y =np.where(Y> rating_pctile[1], 3, Y)

In [600]:
Y = pd.DataFrame(Y , columns=["bean_grade"])
Y

bean_grade
0            3.0
1            3.0
2            3.0
3            3.0
4            3.0
...          ...
1007         1.0
1008         1.0
1009         1.0
1010         1.0
1011         1.0

[1012 rows x 1 columns]

In [601]:
count =  Y['bean_grade'].value_counts()

In [602]:
fig = px.bar(count, text_auto=True , title="Visualize Bar Graph of Number of Samples for each Bean Grade")
fig.show()

# Prepare Train / Test Split

In [763]:
X.shape

(1012, 45)

In [764]:
Y.shape

(1012, 1)

In [765]:
Rseed = 40
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state = Rseed)

In [766]:
x_train

Aroma  Aftertaste   Acidity   Balance  Uniformity  Moisture  \
266  0.040267    0.340962  0.985847  0.497720    0.293848  0.121636   
492  0.872056   -0.170957  0.464930 -0.001828    0.293848  0.579691   
810  0.040267   -1.164682  0.464930 -0.001828   -1.214987  0.808719   
825 -1.323865   -0.652763 -0.349003 -1.236004    0.293848 -2.168642   
566  0.339711   -0.170957 -0.642018 -0.001828    0.293848  0.350663   
..        ...         ...       ...       ...         ...       ...   
440  0.339711   -0.170957 -0.642018  0.497720    0.293848  0.121636   
165  0.339711    0.852881  0.725388  0.732802    0.293848  0.350663   
7    1.703844    2.840331  2.906727  1.702512    0.293848 -2.168642   
219  0.040267    1.093784  0.464930  0.497720    0.293848 -2.168642   
326 -0.225905   -0.170957  0.464930  0.497720    0.293848  0.350663   

     altitude_mean_meters  Species_Robusta  Country.of.Origin_Burundi  \
266             -0.045674                0                          0   
492             -0.055533                0                          0   
810             -0.051191                0                          0   
825              0.219125                0                          0   
566             -0.004299                0                          0   
..                    ...              ...                        ...   
440             -0.078774                0                          0   
165             -0.036847                0                          0   
7                0.014237                0                          0   
219             -0.007058                0                          0   
326             -0.067741                0                          0   

     Country.of.Origin_China  ...  Country.of.Origin_Thailand  \
266                        0  ...                           0   
492                        0  ...                           0   
810                        0  ...                           0   
825                        0  ...                           0   
566                        0  ...                           0   
..                       ...  ...                         ...   
440                        1  ...                           0   
165                        0  ...                           0   
7                          0  ...                           0   
219                        0  ...                           0   
326                        1  ...                           0   

     Country.of.Origin_Uganda  Country.of.Origin_United States  \
266                         0                                0   
492                         0                                0   
810                         0                                0   
825                         0                                0   
566                         0                                0   
..                        ...                              ...   
440                         0                                0   
165                         0                                0   
7                           0                                1   
219                         0                                0   
326                         0                                0   

     Country.of.Origin_United States (Puerto Rico)  Country.of.Origin_Vietnam  \
266                                              0                          0   
492                                              0                          0   
810                                              0                          0   
825                                              0                          0   
566                                              0                          0   
..                                             ...                        ...   
440                                              0                          0   
165                                              0      

In [767]:
X["Aftertaste"] 

0       3.864168
1       3.352249
2       3.111346
3       2.599428
4       3.352249
          ...   
1007    0.581865
1008    1.093784
1009   -0.411860
1010    0.100059
1011    0.100059
Name: Aftertaste, Length: 1012, dtype: float64

In [768]:
plt.figure(figsize=(16,10))
sns.scatterplot(data=X["Aftertaste"] ,color="r")
sns.scatterplot(data=x_train["Aftertaste"] ,color="r")
sns.scatterplot(data=x_test["Aftertaste"] ,color="b")

<AxesSubplot:ylabel='Aftertaste'>

# 7.2 MODEL PREPARATION

## 7.2 (a) KNN Model Training and Testing

In [769]:
# KNN parameter
k = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 25, 35] # try at least 3 values

In [772]:
modelKNN = KNeighborsClassifier(n_neighbors=19, p=2)
modelKNN.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=19)

In [773]:
y_pred= modelKNN.predict(x_test)
KNNScore = accuracy_score(y_test, y_pred)
KNNScore

0.9011976047904192

In [775]:
test_error_rate= []
for k_value in k :
    knn_model = KNeighborsClassifier(n_neighbors=k_value)
    knn_model.fit(x_train, y_train)
    
    y_pred_test = knn_model.predict(x_test)
    
    test_error= accuracy_score(y_test, y_pred_test)
    test_error_rate.append(test_error)

In [776]:
test_error_rate = np.array(test_error_rate).reshape(-1,1)
k = np.array(k).reshape(-1,1)

In [777]:
knn_error_rate = pd.DataFrame(test_error_rate ,columns=["error_rate"])
knn_error_rate["k_values"] = k

In [778]:
fig = px.line(knn_error_rate ,x="k_values" ,y="error_rate", text="k_values" ,markers=True )
fig.update_traces(textposition="bottom right" )
fig.show()

In [792]:
from mlxtend.plotting import plot_decision_regions
#Plotting decision boundaries
plot_decision_regions(x_train, y_train, clf=knn, legend=1)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('K-NN Decision Boundaries')
plt.show()

ModuleNotFoundError: No module named 'mlxtend'

In [779]:
justplot = x_test

In [780]:
justplot['pred'] = y_pred

In [783]:
x_test

Aroma  Aftertaste   Acidity   Balance  Uniformity  Moisture  \
994 -4.118674   -3.182244 -2.269883 -2.940344    0.293848  0.350663   
323  0.040267   -0.923779 -0.642018  3.171770    0.293848 -0.565448   
987 -1.623309   -1.676600 -0.088544 -1.970634   -4.210136  0.579691   
538 -0.225905    0.340962 -0.088544 -0.001828    0.293848  0.350663   
459  0.040267    0.340962  0.171914  0.233254    0.293848  0.579691   
..        ...         ...       ...       ...         ...       ...   
852 -0.791521   -1.164682 -1.162935 -0.971538    0.293848  0.350663   
844 -0.492077   -0.923779 -1.716409 -0.501375    0.293848 -2.168642   
935 -0.791521   -3.182244 -0.902477 -1.471086    0.293848  0.579691   
166  1.171499    1.334687  1.539321  1.232349    0.293848 -0.107392   
450 -0.225905    0.100059  0.464930  0.233254    0.293848 -2.168642   

     altitude_mean_meters  Species_Robusta  Country.of.Origin_Burundi  \
994             -0.067741                0                          0   
323             -0.029124                0                          0   
987             -0.012574                0                          0   
538             -0.133940                0                          0   
459             -0.029124                0                          0   
..                    ...              ...                        ...   
852             -0.055533                0                          0   
844              0.272857                0                          0   
935             -0.055533                0                          0   
166             -0.001541                0                          0   
450             -0.020849                0                          0   

     Country.of.Origin_China  ...  Country.of.Origin_Uganda  \
994                        0  ...                         0   
323                        0  ...                         0   
987                        0  ...                         0   
538                        0  ...                         0   
459                        0  ...                         0   
..                       ...  ...                       ...   
852                        0  ...                         0   
844                        0  ...                         0   
935                        0  ...                         0   
166                        0  ...                         0   
450                        0  ...                         0   

     Country.of.Origin_United States  \
994                                0   
323                                0   
987                                0   
538                                0   
459                                0   
..                               ...   
852                                0   
844                                0   
935                                0   
166                                0   
450                                0   

     Country.of.Origin_United States (Puerto Rico)  Country.of.Origin_Vietnam  \
994                                              0                          0   
323                                              0                          1   
987                                              0                          0   
538                                              0                          0   
459                                              0                          0   
..                                             ...                        ...   
852                                              0                          0   
844                                              0                          0   
935                                              0                          0   
166                                              0                          0   
450                                              0                          0   

     Country.of.Origin_Zambia  Processing.Method_Other  \
994      

In [790]:
fig = px.scatter_3d(justplot, x='Balance', y='Aftertaste',z="Acidity",
              color="pred" , symbol='pred')
fig.show()